In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd 
import matplotlib.pyplot as plt
import seaborn as sns
from mysql.connector import connect
from typing import List
import time, random
from urllib.parse import urljoin
import os

Retrieving 10 pages data of machanical keyboard from Amazon using BeautifulSoup Library

In [2]:
def get_body(url):
    print("Connecting...")
    header = {
            "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8",
            "Accept-Language": "en-US,en;q=0.9",
            "Connection": "keep-alive",
            "User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/17.6 Safari/605.1.15"

            }
    response = requests.get(url,headers=header)
    if response.status_code == 200:
        html_body = response.content
        soup = BeautifulSoup(html_body,'html.parser')
    else:
        soup = f"Fail! Could not connect with url: {response.status_code}"
    return soup

def get_links(url):
    soup = get_body(url)
    list_link = []
    links = soup.find_all("a",attrs={"class":"a-link-normal s-underline-text s-underline-link-text s-link-style a-text-normal"})
    list_link = [link["href"] for link in links]
    return list_link

def get_data(links:List) -> pd.DataFrame:
    all_product_data =[]

    #exact
    base_url = "https://www.amazon.com"
    for link in links:
 
        #initilize data dict 
        product_data = {
        "Title": None,
        "Link":None,
        "Brand": None,
        "Price":None,
        "Review":None,
        "Total Review":None,
        "Compatible Devices": None,
        "Color": None,
        "Number of Keys": None,
        "Connectivity Technology": None,
        "Rank":None
                    }
        time.sleep(random.uniform(1, 3))
        new_url = urljoin(base_url, link)
        soup = get_body(new_url)

        product_data["Link"] =  new_url

        #product title
        h1 = soup.find("span",attrs={"id":"productTitle"})
        try:
            h1 = soup.find("span", attrs={"id": "productTitle"})
            product_data["Title"] = h1.text.strip() if h1 else None
        except AttributeError:
            product_data["Title"] = None
        else:
            print("Complete title part")
            
        ##review
        review_container = soup.find("div",attrs={"id":"customerReviews"})
        try:
            review = review_container.find("span",attrs={"data-hook":"rating-out-of-text"})
            product_data["Review"] = review.text
        except AttributeError:
            product_data["Review"] = None
        else:
            print("Complete review part")


        #total review
        try:
            review_total = review_container.find('span',attrs={"data-hook":"total-review-count"}).text
            product_data["Total Review"] = review_total
        except AttributeError:
            product_data["Total Review"] = None
        else:
            print('Complete total review part')

        ##price
        try:
            price_symbol = soup.find(class_='a-offscreen').text
            product_data["Price"] = price_symbol
        except AttributeError:
            product_data["Price"] = None
        else:
            print("Complete price part")

        # print(product_detail)
        product_detail = soup.find(class_="a-normal a-spacing-micro")
        try:
            for row in product_detail.find_all('tr'):
                name = row.find('td',attrs={"class":"a-span3"}).text.strip()
                feature = row.find('td',attrs={"class":"a-span9"}).text
                if name in product_data:
                    product_data[name] = feature
        except AttributeError:
            product_data[name] = None
        else:
            print("Complete")
            # print(name.text, "-", feature.text)

        #rank
        addition_value = soup.find("table",attrs={"id":"productDetails_detailBullets_sections1"})
        try:
            for row in addition_value.find_all('tr'):
                header = row.find('th').text
                if "Best Sellers Rank" in header:
                    rank = row.find("td").text
                    product_data["Rank"] = rank
        except AttributeError:
            product_data["Rank"] = None
        else:
            print("Complete rank part")
        
        print(">>Finish link %s"%link)
        
        all_product_data.append(product_data)
    else:
        data = pd.DataFrame(all_product_data)
        return data
    
def retrieve_page_data(start,end=None):
    df=[]
    if end is None:
        end = start
        start = 1
    
    for i in range(start,end):
        url=f"https://www.amazon.com/s?k=Computer+Keyboards&i=computers&rh=n%3A12879431&page=2&c=ts&qid=1724479075&ts_id=12879431&ref=sr_pg_{i}"
        list_links = get_links(url)
        data = get_data(list_links)
        df.append(data)
    combine = pd.concat(df,ignore_index=True)

    print(f"Finshed retrieving data from page {start} to {end-1}")
    return combine

def combine_all_df(*args) -> pd.DataFrame:
    df = pd.concat(args,ignore_index=True)

    print(f"Finish combining {len(args)} DataFrames")
    return df

def write_csv(foldername,filename,data:pd.DataFrame):
    present_path = os.getcwd()
    folder_path = os.path.join(present_path,foldername)

    if not os.path.exists(folder_path):
        os.makedirs(folder_path)
        print(f"Folder {foldername} is created in {present_path}")
    
    file_path = os.path.join(folder_path,filename)
    data.to_csv(file_path, index=True)
    print(f"Data saved to {file_path}")


In [3]:
#fetch data of first 2 pages
data_part1 = retrieve_page_data(3)

Connecting...
Connecting...
Complete title part
Complete review part
Complete total review part
Complete price part
Complete
Complete rank part
>>Finish link /sspa/click?ie=UTF8&spc=MTozMzA5Mjc2MTE4MTU0OTIxOjE3MjQ4NDc3NDA6c3BfYXRmX25leHQ6MzAwMjk1NDcwNTIyNjAyOjowOjo&url=%2FLIZRROT-Wireless-Ergonomic-Keyboard-Comfortable%2Fdp%2FB0D46S3PRQ%2Fref%3Dsr_1_25_sspa%3Fc%3Dts%26dib%3DeyJ2IjoiMSJ9.6nxUPKjuMbQABbZomCNndSlt4aXBasGjJuS-jwLb2VsIjQyhKq5xBmpM5czsHD1BMoBgcjIskw5GGF3sYDAi-ujUItPdMVXL-z3JCY17rKJn9-GSIRdLTJU_XqurLXJzT0LXIpCtq1wiFinbDs5Br_QQBmzA3qrOEpHFc3l3sv1QdrBu2Osb2MLw-HsZpkbBZ6Uws16tWcgpts_pHb8RPZtH8c_x00ChkmUum1VwNgvr0nSetNgxi9mHxetDxgNQs3oqtnOEdrs09WAb137GzlJ7GgsER9pWohbX7yx7ws8.zSX6KfI8qEqlmjRvev1IIPgSmTV-N-W-aykbRL30WQE%26dib_tag%3Dse%26keywords%3DComputer%2BKeyboards%26qid%3D1724847740%26s%3Dpc%26sr%3D1-25-spons%26ts_id%3D12879431%26sp_csd%3Dd2lkZ2V0TmFtZT1zcF9hdGZfbmV4dA%26psc%3D1
Connecting...
Complete title part
Complete review part
Complete total review part
Complete price par

In [4]:
data_part1.shape

(62, 11)

In [5]:
data_part2=retrieve_page_data(3,5)

Connecting...
Connecting...
Complete title part
Complete review part
Complete total review part
Complete price part
Complete
Complete rank part
>>Finish link /sspa/click?ie=UTF8&spc=MToxNjM4Mjg5NDI2OTAyOTc0OjE3MjQ4NDgwNzA6c3BfYXRmX25leHQ6MzAwMTA3NDM5OTU1ODAyOjowOjo&url=%2FMARVO-Keyboard-Computer-Backlight-Full-Size%2Fdp%2FB0CH9QTXXK%2Fref%3Dsr_1_25_sspa%3Fc%3Dts%26dib%3DeyJ2IjoiMSJ9.6nxUPKjuMbQABbZomCNndSlt4aXBasGjJuS-jwLb2VsIjQyhKq5xBmpM5czsHD1BMoBgcjIskw5GGF3sYDAi-ujUItPdMVXL-z3JCY17rKKWORaP8WVQ6iwgecMDsKSVHdYsLdVZqbKmr87JkJSs1fQQBmzA3qrOEpHFc3l3sv1QdrBu2Osb2MLw-HsZpkbBZ6Uws16tWcgpts_pHb8RPYN2K_Iljp5EFnKm3oJ4eGOmNQuO3Qdw-8R6f2OQ9Y52s3oqtnOEdrs09WAb137GzlJ7GgsER9pWohbX7yx7ws8.upMYFcv180sxeNkN9x1_NTqD376geZNYGVaNjUve4Ic%26dib_tag%3Dse%26keywords%3DComputer%2BKeyboards%26qid%3D1724848070%26s%3Dpc%26sr%3D1-25-spons%26ts_id%3D12879431%26sp_csd%3Dd2lkZ2V0TmFtZT1zcF9hdGZfbmV4dA%26psc%3D1
Connecting...
Complete title part
Complete review part
Complete total review part
Complete price part
Co

In [6]:
data_part2.shape

(62, 11)

In [7]:
data_part3 = retrieve_page_data(5,7)

Connecting...
Connecting...
Complete title part
Complete review part
Complete total review part
Complete price part
Complete
Complete rank part
>>Finish link /sspa/click?ie=UTF8&spc=MTo0MjU0NDA0NDM0ODY1ODQ1OjE3MjQ4NDgzOTU6c3BfYXRmX25leHQ6MzAwMTc0ODE5NTM3NTAyOjowOjo&url=%2FBRJEC-Wireless-Ergonomic-Bluetooth-Compatible%2Fdp%2FB0BCYRSZYB%2Fref%3Dsr_1_25_sspa%3Fc%3Dts%26dib%3DeyJ2IjoiMSJ9.6nxUPKjuMbQABbZomCNndSlt4aXBasGjJuS-jwLb2VsIjQyhKq5xBmpM5czsHD1BMoBgcjIskw5GGF3sYDAi-ujUItPdMVXL-z3JCY17rKJn9-GSIRdLTJU_XqurLXJzT0LXIpCtq1wiFinbDs5Br_QQBmzA3qrOEpHFc3l3sv1QdrBu2Osb2MLw-HsZpkbBZ6Uws16tWcgpts_pHb8RPZtH8c_x00ChkmUum1VwNgvr0nSetNgxi9mHxetDxgNQs3oqtnOEdrs09WAb137GzlJ7GgsER9pWohbX7yx7ws8.e9VkctEFiASUvL-bl-5M7DP3XW3Jkg1MYD3nVdiKqzM%26dib_tag%3Dse%26keywords%3DComputer%2BKeyboards%26qid%3D1724848394%26s%3Dpc%26sr%3D1-25-spons%26ts_id%3D12879431%26sp_csd%3Dd2lkZ2V0TmFtZT1zcF9hdGZfbmV4dA%26psc%3D1
Connecting...
Complete title part
Complete review part
Complete total review part
Complete price part


In [8]:
data_part3.shape

(62, 11)

In [9]:
data_part4 = retrieve_page_data(7,9)

Connecting...
Connecting...
Complete title part
Complete review part
Complete total review part
Complete price part
Complete
Complete rank part
>>Finish link /sspa/click?ie=UTF8&spc=MTo0ODE0MzI3NTU1MTA5MjI2OjE3MjQ4NDg3MjU6c3BfYXRmX25leHQ6MzAwMzgyNjY2ODI2MzAyOjowOjo&url=%2FMOFII-Computer-Keyboards-Typewriter-Aesthetic%2Fdp%2FB0CXSC4Y2K%2Fref%3Dsr_1_25_sspa%3Fc%3Dts%26dib%3DeyJ2IjoiMSJ9.xdo1IDiXOV9O_fehgFL7ec9oPgpcIUDOAdydmZTsnYc9xxkwzK-CuIFuM0xnJzSvDVw06w1Ff2T2MHVFkBPqToH99PoPfEXv4fKGzZ-Kko7Us0ydKUIuFr7EBLkGVeuuHdYsLdVZqbKmr87JkJSs1fQQBmzA3qrOEpHFc3l3sv1QdrBu2Osb2MLw-HsZpkbBZ6Uws16tWcgpts_pHb8RPYN2K_Iljp5EFnKm3oJ4eGOmNQuO3Qdw-8R6f2OQ9Y52s3oqtnOEdrs09WAb137GzlJ7GgsER9pWohbX7yx7ws8.Kc95drM2pDswXe1rGHakfmra1N45YgPGdg-ra0LiHVE%26dib_tag%3Dse%26keywords%3DComputer%2BKeyboards%26qid%3D1724848724%26s%3Dpc%26sr%3D1-25-spons%26ts_id%3D12879431%26sp_csd%3Dd2lkZ2V0TmFtZT1zcF9hdGZfbmV4dA%26psc%3D1
Connecting...
Complete title part
Complete review part
Complete total review part
Complete price part


In [10]:
data_part4.shape

(62, 11)

In [11]:
data_part5 = retrieve_page_data(9,11)

Connecting...
Connecting...
Complete title part
Complete review part
Complete total review part
Complete price part
Complete
Complete rank part
>>Finish link /sspa/click?ie=UTF8&spc=MToxNDg1NTg3MDA4MDU2Njg0OjE3MjQ4NDkwNTA6c3BfYXRmX25leHQ6MzAwMjk1NDcwNTIyNjAyOjowOjo&url=%2FLIZRROT-Wireless-Ergonomic-Keyboard-Comfortable%2Fdp%2FB0D46S3PRQ%2Fref%3Dsr_1_25_sspa%3Fc%3Dts%26dib%3DeyJ2IjoiMSJ9.Xyc35cRErMZpEQxIn0vK3XYGQPeO0t-k7cE44smJDVO0wI0CbaWa9G0bsM4Q9Ch6DVw06w1Ff2T2MHVFkBPqTiyF0etU_-haKzHS5W31pSL8ybFFsdNgRhCriTMXjdCAseFoPI_qltKwRTlU8vkrqkEaqN8HnTv5P6h-ggIzQ8Dn-ngTo-C9DJ6PONluU7D_p1ww1F4z1HWXuzIyEEpeVBunOrwswfwYH5x66Qx4GilFTrbkFCpewP0VO9vzOLR5k5RrSx1_taSZJI7SE23UX_ed79JbDahW1GOLe3WH9ts.os7yplUuPv5jQ65GV1Kn8yQmuELkdjv5wUaIHPAFKOk%26dib_tag%3Dse%26keywords%3DComputer%2BKeyboards%26qid%3D1724849050%26s%3Dpc%26sr%3D1-25-spons%26ts_id%3D12879431%26sp_csd%3Dd2lkZ2V0TmFtZT1zcF9hdGZfbmV4dA%26psc%3D1
Connecting...
Complete title part
Complete review part
Complete total review part
Complete price par

In [12]:
data_part5.shape

(62, 11)

In [13]:
data = combine_all_df(data_part1,data_part2,data_part3,data_part4,data_part5)
write_csv("Data","Products",data)

Finish combining 5 DataFrames
Data saved to /Users/huyenhoang/Documents/Data-Projetcs/Amazon Project/Crawl data/Data/Products


Load data to database

In [19]:
# create functions for interaction with database
def connect_to_mysql(host,user,password,port,database = None):
    try:
        if database:
            conn= connect(host=host, user=user, password=password,port=port,database=database)
        else:
            conn = connect(host=host, user=user, password=password,port=port)
        return conn
    except Exception as e:
        print(f"Error: {e}")

def create_database(cursor,db):
    database_exists = False
    try:
        cursor.execute(f"SHOW DATABASES")
        databases = cursor.fetchall()
        for database in databases:
            if db in database:
                database_exists = True
                break
        if database_exists:
            print("Database exists")
        else:
            cursor.execute(f"CREATE DATABASE {db}")
            print(f"Create successfully database {db}")
    except Exception as e:
        print(f"Error: {e}")

def create_table(conn,cursor,table_name):
    sql = f"""
    CREATE TABLE IF NOT EXISTS {table_name} (
        id INT AUTO_INCREMENT PRIMARY KEY,
        title VARCHAR(255),
        link TEXT,
        brand VARCHAR(255),
        price VARCHAR(255),
        review VARCHAR(255),
        total_review VARCHAR(255),
        compatible_devices TEXT,
        color VARCHAR(255),
        numkeys INT,
        connect_tech VARCHAR(255),
        rank_amz VARCHAR(255)
        )
    """

    try:
        cursor.execute(sql)
        conn.commit()
        print(f"Table {table_name} is created successfully")
    except Exception as e:
        conn.rollback()
        print(f"Error: {e}")

def transfer_data(conn,cursor,df, table_name):
    insert_sql = f"""
        INSERT INTO {table_name}(title, link, brand, price, review, total_review, compatible_devices, color, numkeys, connect_tech, rank_amz) 
        VALUE(%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s)     
        """
    data_to_insert = df.values.tolist()
    try:
        cursor.executemany(insert_sql,data_to_insert)
        conn.commit()
        print(f"Data inserted into table {table_name}")
    except Exception as e:
        conn.rollback()
        print(f"Error: {e}")


In [16]:
host = "root"
user = "root"
password = "root"
port = "3306"

#create connection
conn = connect_to_mysql(host=host,user=user,password=password,port=port)
conn.is_connected()

True

In [17]:
cursor = conn.cursor()
create_database(cursor,"amazon")

Create successfully database amazon


In [20]:
conn.database = "amazon"
create_table(conn,cursor,"keyboards")

Table keyboards is created successfully


In [21]:
#transfer data
transfer_data(conn,cursor,data,"keyboards")

Data inserted into table keyboards
